In [1]:
# pseduo code
import rasterio
from rasterio.mask import mask
from rasterio.windows import from_bounds
from rasterio.transform import from_bounds 
import numpy as np
import sys
import os
import subprocess
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.utils import register_keras_serializable
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Input, Add
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

start_time = time.time()
print("modules established")

## establish file paths
years = list(range(2022, 2025))
Domain = "Sierras"
WorkspaceBase = "/pl/active/hydroServer/ASO_DeepLearningData/CNN_v.02/"
phv_features = WorkspaceBase + "features/scaled/"
tree_workspace = WorkspaceBase + "treeCover/"
modelOuptuts = WorkspaceBase + "modelOutputs/"
final_activation = 'relu'

## seting folder
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# create folder for model outputs
os.makedirs(modelOuptuts + f"{str(timestamp)}/", exist_ok=True)
inter_model_outWorkspace = modelOuptuts + f"{str(timestamp)}/"


modules established


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/'

In [ ]:
## function for min-max scaling
def min_max_scale(data, min_val=None, max_val=None, feature_range=(0, 1)):
    """Min-Max normalize a NumPy array to a target range."""
    data = data.astype(np.float32)
    mask = np.isnan(data)

    d_min = np.nanmin(data) if min_val is None else min_val
    d_max = np.nanmax(data) if max_val is None else max_val

    # if d_max == d_min:
    #     raise ValueError("Min and max are equal — can't scale.")
    if d_max == d_min:
        return np.full_like(data, feature_range[0], dtype=np.float32)

    a, b = feature_range
    scaled = (data - d_min) / (d_max - d_min)  # to [0, 1]
    scaled = scaled * (b - a) + a              # to [a, b]

    scaled[mask] = np.nan  # preserve NaNs
    return scaled

In [ ]:


def read_aligned_raster(src_path, extent, target_shape, nodata_val=-1):
    height, width = target_shape
    transform = from_bounds(*extent, width=width, height=height)

    with rasterio.open(src_path) as src:
        try:
            data = src.read(
                1,
                out_shape=target_shape,
                resampling=rasterio.enums.Resampling.nearest,
                window=src.window(*extent)
            )
        except Exception as e:
            print(f"Failed to read {src_path}: {e}")
            return np.full(target_shape, nodata_val, dtype=np.float32)

        # Handle nodata in source
        src_nodata = src.nodata
        if src_nodata is not None:
            data = np.where(data == src_nodata, np.nan, data)

        # Replace NaNs or invalid with -1
        data = np.where(np.isnan(data), nodata_val, data)

        return data

In [ ]:

def save_array_as_raster(output_path, array, extent, crs, nodata_val=-1):
    height, width = array.shape
    transform = from_bounds(*extent, width=width, height=height)
    
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=array.dtype,
        crs=crs,
        transform=transform,
        nodata=nodata_val
    ) as dst:
        dst.write(array, 1)

In [ ]:
# split up the features and arrarys 

## create empty arrays
featureArray = []
targetArray = []

# loop through the years and feature data
for year in years:
    print(f"Processing year {year}")
    targetSplits = WorkspaceBase + f"{year}/SWE_processed_splits/"
    fSCAWorkspace = WorkspaceBase + f"{year}/fSCA/"
    for sample in os.listdir(targetSplits):
        featureTuple = ()
        featureName = []
        # loop through each sample and get the corresponding features
        if sample.endswith("nonull_fnl.tif"):
            # read in data
            with rasterio.open(targetSplits + sample) as samp_src:
                samp_data = samp_src.read(1)
                meta = samp_src.meta.copy()
                samp_extent = samp_src.bounds
                samp_transform = samp_src.transform
                samp_crs = samp_src.crs
    
                # apply a mask to all no data values. Reminder that nodata values is -9999
                mask = samp_data >= 0
                msked_target = np.where(mask, samp_data, -1)
                target_shape = msked_target.shape
    
                # flatted data
                samp_flat = msked_target.flatten()
                

            # try to get the fsca variables 
            sample_root = "_".join(sample.split("_")[:2])
            for fSCA in os.listdir(fSCAWorkspace):
                if fSCA.endswith(".tif") and fSCA.startswith(sample_root):
                    featureName.append(f"{fSCA[:-4]}")
                    fsca_norm = read_aligned_raster(src_path=fSCAWorkspace + fSCA, extent=samp_extent, target_shape=target_shape)
                    fsca_norm = min_max_scale(fsca_norm, min_val=0, max_val=100)
                    featureTuple += (fsca_norm,)
                    # print(fsca_norm.shape)
                    fsca_flat = fsca_norm.flatten()
                    fsca_binary_mask = (fsca_flat > 0).astype(np.float32)
                    if fsca_norm.shape != (256, 256):
                        print(f"WRONG SHAPE FOR {sample}: FSCA")
                        output_debug_path = f"./debug_output/{sample_root}_BAD_FSCA.tif"
                        save_array_as_raster(
                            output_path=output_debug_path,
                            array=fsca_norm,
                            extent=samp_extent,
                            crs=samp_crs,
                            nodata_val=-1
                        )
    
            # get a DOY array into a feature 
            date_string = sample.split("_")[1]
            doy_str = date_string[-3:]
            doy = float(doy_str)
            DOY_array = np.full_like(msked_target, doy)
            doy_norm = min_max_scale(DOY_array,  min_val=0, max_val=366)
            featureTuple += (doy_norm,)
            featureName.append(doy)
    
            # get the vegetation array
            for tree in os.listdir(tree_workspace):
                if tree.endswith(".tif"):
                    if tree.startswith(f"{year}"):
                        featureName.append(f"{tree[:-4]}")
                        tree_norm = read_aligned_raster(
                        src_path=tree_workspace + tree,
                        extent=samp_extent,
                        target_shape=target_shape
                        )
                        tree_norm = min_max_scale(tree_norm, min_val=0, max_val=100)
                        featureTuple += (tree_norm,)
                        if tree_norm.shape != (256, 256):
                            print(f"WRONG SHAPE FOR {sample}: TREE")
                            output_debug_path = f"./debug_output/{sample_root}_BAD_TREE.tif"
                            save_array_as_raster(
                                output_path=output_debug_path,
                                array=fsca_norm,
                                extent=samp_extent,
                                crs=samp_crs,
                                nodata_val=-1
                            )
            
            # # get all the features in the fodler 
            for phv in os.listdir(phv_features):
                if phv.endswith(".tif"):
                    featureName.append(f"{phv[:-4]}")
                    phv_data = read_aligned_raster(src_path=phv_features + phv, extent=samp_extent, target_shape=target_shape)
                    featureTuple += (phv_data,)
                    if phv_data.shape != (256, 256):
                         print(f"WRONG SHAPE FOR {sample}: {phv}")
                        
            feature_stack = np.dstack(featureTuple)
            if feature_stack.shape[2] != 14:
                print(f"{sample} has shape {feature_stack.shape} — missing or extra feature?")
                print(featureName)
                print(" ")
            else:
                featureArray.append(feature_stack)
                target_combined = np.concatenate([samp_flat, fsca_binary_mask])
                targetArray.append(target_combined)
                # targetArray.append(samp_flat)
    print("You go girl!")
X = np.array(featureArray)
y = np.array(targetArray)
print("all data split into target and feature array")



In [ ]:
@register_keras_serializable()
def masked_loss_fn(y_true, y_pred, loss_fn, mask_value=-1):
    # Use tf.shape instead of .shape attribute for graph compatibility
    y_true_shape = tf.shape(y_true)[1]
    y_pred_shape = tf.shape(y_pred)[1]
    
    # Safely extract SWE part using dynamic slicing
    y_true_swe = tf.cond(
        tf.greater(y_true_shape, y_pred_shape),
        lambda: y_true[:, :y_pred_shape],
        lambda: y_true
    )
    
    # Create mask for valid values
    mask = tf.not_equal(y_true_swe, mask_value)
    
    # Use boolean_mask with safety checks
    y_true_masked = tf.boolean_mask(y_true_swe, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    
    # Apply the loss function
    return loss_fn(y_true_masked, y_pred_masked)

@register_keras_serializable(name="masked_mse")
def masked_mse(y_true, y_pred):
    # Create loss function instance outside the function call for stability
    mse_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    return masked_loss_fn(y_true, y_pred, mse_fn, mask_value=-1)

@register_keras_serializable(name="masked_mae")
def masked_mae(y_true, y_pred):
    mae_fn = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)
    return masked_loss_fn(y_true, y_pred, mae_fn, mask_value=-1)

@register_keras_serializable(name="masked_rmse")
def masked_rmse(y_true, y_pred):
    mse_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    mse = masked_loss_fn(y_true, y_pred, mse_fn, mask_value=-1)
    return tf.sqrt(mse)
    
@register_keras_serializable(name="constrained_masked_mse")
def constrained_masked_mse(mask_value=-1, penalty_weight=10.0):
    def loss(y_true, y_pred):
        # Get dynamic shapes
        y_true_shape = tf.shape(y_true)
        batch_size = y_true_shape[0]
        
        # Extract SWE values and fSCA mask from y_true
        half_size = 65536  # 256*256
        swe_true = y_true[:, :half_size]  # First half is SWE values
        fsca_mask = y_true[:, half_size:]  # Second half is fSCA mask
        
        # Create mask for valid SWE values (not equal to mask_value)
        valid_mask = tf.not_equal(swe_true, mask_value)
        
        # Apply masking to calculate base MSE loss
        swe_true_masked = tf.boolean_mask(swe_true, valid_mask)
        y_pred_masked = tf.boolean_mask(y_pred, valid_mask)
        
        # Base MSE loss
        mse = tf.reduce_mean(tf.square(swe_true_masked - y_pred_masked))
        
        # Reshape predictions and fSCA mask to 2D images for constraint calculation
        y_pred_img = tf.reshape(y_pred, [batch_size, 256, 256])
        fsca_img = tf.reshape(fsca_mask, [batch_size, 256, 256])
        
        # Calculate constraint violation: SWE <= 0 where fSCA > 0
        violation_mask = tf.logical_and(fsca_img > 0, y_pred_img <= 0)
        
        # Count violations relative to number of fSCA pixels
        violation_count = tf.reduce_sum(tf.cast(violation_mask, tf.float32), axis=[1, 2])
        total_fsca_pixels = tf.reduce_sum(tf.cast(fsca_img > 0, tf.float32), axis=[1, 2])
        
        # Avoid division by zero with a small epsilon
        epsilon = 1e-7
        safe_total = total_fsca_pixels + epsilon
        
        # Calculate violation ratio
        violation_ratio = violation_count / safe_total
        
        # Apply penalty
        penalty = tf.reduce_mean(violation_ratio) * penalty_weight
        
        # The final loss is MSE + penalty
        return mse + penalty
    
    return loss

@register_keras_serializable(name="constraint_violation_ratio")
def constraint_violation_ratio(y_true, y_pred):
    # Get dynamic shapes
    batch_size = tf.shape(y_true)[0]
    half_size = 65536  # 256*256
    
    # Extract SWE and fSCA from y_true
    fsca_mask = y_true[:, half_size:]
    
    # Reshape to 2D images
    y_pred_img = tf.reshape(y_pred, [batch_size, 256, 256])
    fsca_img = tf.reshape(fsca_mask, [batch_size, 256, 256])
    
    # Calculate violation ratio
    violation_mask = tf.logical_and(fsca_img > 0, y_pred_img <= 0)
    violation_count = tf.reduce_sum(tf.cast(violation_mask, tf.float32), axis=[1, 2])
    total_fsca_pixels = tf.reduce_sum(tf.cast(fsca_img > 0, tf.float32), axis=[1, 2])
    
    # Avoid division by zero with small epsilon
    epsilon = 1e-7
    safe_total = total_fsca_pixels + epsilon
    
    return tf.reduce_mean(violation_count / safe_total)

In [ ]:
class SNOWDataGenerator(Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True, augment=False, mask_value=-1):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mask_value = mask_value
        self.indices = np.arange(len(X))
        
        # Verify data shapes
        print(f"X shape: {X.shape}")
        print(f"y shape: {y.shape}")
        
        # Make sure y has the right shape (each sample should be [SWE, fSCA])
        if len(y.shape) == 2 and y.shape[1] == 65536 * 2:
            print("Target data has correct shape for SWE + fSCA mask")
        else:
            print(f"WARNING: Target data shape {y.shape} might not be compatible")
            
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, index):
        # Get batch indices
        start_idx = index * self.batch_size
        end_idx = min((index + 1) * self.batch_size, len(self.X))
        batch_indices = self.indices[start_idx:end_idx]
        
        # Fetch batch data
        X_batch = self.X[batch_indices].copy()
        y_batch = self.y[batch_indices].copy()
        
        # Apply augmentation if enabled
        if self.augment:
            X_batch, y_batch = self._augment_batch(X_batch, y_batch)
        
        # Debug: Print shapes before returning
        # print(f"Batch X shape: {X_batch.shape}, y shape: {y_batch.shape}")
            
        return X_batch, y_batch
    
    def _augment_batch(self, X_batch, y_batch):
        """Apply data augmentation to the batch"""
        for i in range(len(X_batch)):
            # 50% chance to apply an augmentation
            if np.random.random() > 0.5:
                # Choose a random transformation
                choice = np.random.randint(0, 3)
                
                if choice == 0:  # Horizontal flip
                    X_batch[i] = X_batch[i, :, ::-1, :]
                    # Split y and handle separately
                    swe = y_batch[i, :65536].reshape(256, 256)
                    fsca = y_batch[i, 65536:].reshape(256, 256)
                    swe = swe[:, ::-1]
                    fsca = fsca[:, ::-1]
                    y_batch[i, :65536] = swe.flatten()
                    y_batch[i, 65536:] = fsca.flatten()
                    
                elif choice == 1:  # Vertical flip
                    X_batch[i] = X_batch[i, ::-1, :, :]
                    swe = y_batch[i, :65536].reshape(256, 256)
                    fsca = y_batch[i, 65536:].reshape(256, 256)
                    swe = swe[::-1, :]
                    fsca = fsca[::-1, :]
                    y_batch[i, :65536] = swe.flatten()
                    y_batch[i, 65536:] = fsca.flatten()
                    
                elif choice == 2:  # Random 90-degree rotation
                    k = np.random.randint(1, 4)  # 1, 2, or 3 for 90, 180, 270 degrees
                    X_batch[i] = np.rot90(X_batch[i], k=k, axes=(0, 1))
                    swe = y_batch[i, :65536].reshape(256, 256)
                    fsca = y_batch[i, 65536:].reshape(256, 256)
                    swe = np.rot90(swe, k=k)
                    fsca = np.rot90(fsca, k=k)
                    y_batch[i, :65536] = swe.flatten()
                    y_batch[i, 65536:] = fsca.flatten()
        
        return X_batch, y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# to gradually increase the weights
class ConstraintWeightScheduler(tf.keras.callbacks.Callback):
    def __init__(self, initial_weight=1.0, max_weight=100.0, increase_per_epoch=5.0):
        super().__init__()
        self.initial_weight = initial_weight
        self.max_weight = max_weight
        self.increase_per_epoch = increase_per_epoch
        self.current_weight = initial_weight
    
    def on_epoch_begin(self, epoch, logs=None):
        # Calculate new weight
        self.current_weight = min(
            self.initial_weight + epoch * self.increase_per_epoch,
            self.max_weight
        )
        
        # Get the loss function - safely handle custom objects with closures
        loss_fn = self.model.loss
        
        # Since we can't directly update the weight variable,
        # print a message recommending using a LearningRateScheduler instead
        print(f"\nEpoch {epoch+1}: Constraint penalty weight should be {self.current_weight:.2f}")
        print("Note: To dynamically change penalty weight, consider implementing a custom training loop")

class GradientMonitoringModel(tf.keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(
                y, y_pred, regularization_losses=self.losses
            )

        gradients = tape.gradient(loss, self.trainable_weights)

        # Use tf ops, not .numpy() (because we're in graph mode)
        gradient_norms = [tf.norm(g) if g is not None else tf.constant(0.0) for g in gradients]

        max_norm = tf.reduce_max(gradient_norms)
        min_norm = tf.reduce_min(gradient_norms)
        mean_norm = tf.reduce_mean(gradient_norms)

        # Log using tf.print so it works in graph mode
        tf.print(
            "\nGradient norms - Max:", max_norm,
            "Min:", min_norm,
            "Mean:", mean_norm
        )

        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}


In [ ]:
print("shape of input data")
print(f"feature shape: {X.shape}")
print(f"target shape: {y.shape}")

# split between training and test data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, shuffle=True)
print(" ")
print("________________________________ Training and Validation Data Shapes ________________________________")
print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_valid.shape, y_valid.shape)
print("***")

# Build Network
inputs = Input(shape=(256, 256, 14))

x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

x = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), padding='valid')(x)

x = Flatten()(x)
x = Dropout(0.1)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

outputs = Dense(65536, activation=final_activation)(x)

# Wrap in your custom model subclass
model = GradientMonitoringModel(inputs=inputs, outputs=outputs)

In [9]:
# uploading and saving model stats
from tensorflow.keras import backend as K

# Compile the model with the loss function
model.compile(
    optimizer='adam',
    loss=constrained_masked_mse(mask_value=-1, penalty_weight=10.0),
    metrics=[masked_rmse, masked_mae, masked_mse, constraint_violation_ratio]
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    f"{inter_model_outWorkspace}/best_model_{timestamp}.keras",
    monitor="val_loss",  
    verbose=1, save_best_only=True, mode='min'
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    mode='min', verbose=1, patience=10, restore_best_weights=True
)

tensorboard = TensorBoard(
    log_dir=f"{inter_model_outWorkspace}/logs_{timestamp}",
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

# get model summary
model.summary()

Model: "gradient_monitoring_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 14)]    0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 64)      8128      
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     73856     
                                                                 
 batch_normalization_1 (Batc  (None, 125,

In [ ]:
# establish the model
batch_size = 4
train_gen = SNOWDataGenerator(X_train, y_train, batch_size=32, shuffle=True, augment=True)
valid_gen = SNOWDataGenerator(X_valid, y_valid, batch_size=32, shuffle=False, augment=False)

history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[checkpoint, early_stopping, reduce_lr, tensorboard],
)

X shape: (940, 256, 256, 14)
y shape: (940, 131072)
Target data has correct shape for SWE + fSCA mask
X shape: (166, 256, 256, 14)
y shape: (166, 131072)
Target data has correct shape for SWE + fSCA mask
Epoch 1/100

Gradient norms - Max: 0.0753051266 Min: 0.000168458297 Mean: 0.0140555613
 1/30 [>.............................] - ETA: 5:04 - loss: 4.9283 - masked_rmse: 0.4988 - masked_mae: 0.3700 - masked_mse: 0.2488 - constraint_violation_ratio: 0.4680
Gradient norms - Max: 0.178570941 Min: 0.00062865566 Mean: 0.038293913
 2/30 [=>............................] - ETA: 1:31 - loss: 4.9011 - masked_rmse: 0.4650 - masked_mae: 0.3373 - masked_mse: 0.2173 - constraint_violation_ratio: 0.4684
Gradient norms - Max: 0.101657622 Min: 0.000220726652 Mean: 0.0215456113
 3/30 [==>...........................] - ETA: 1:22 - loss: 4.6417 - masked_rmse: 0.4263 - masked_mae: 0.3043 - masked_mse: 0.1855 - constraint_violation_ratio: 0.4456
Gradient norms - Max: 0.255201876 Min: 0.000431962137 Mean: 0.05

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 8))

# Main loss plot
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# RMSE plot
plt.subplot(2, 2, 2)
plt.plot(history.history['masked_rmse'])
plt.plot(history.history['val_masked_rmse'])
plt.title('Masked RMSE')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# MAE plot
plt.subplot(2, 2, 3)
plt.plot(history.history['masked_mae'])
plt.plot(history.history['val_masked_mae'])
plt.title('Masked MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# Constraint violation plot
plt.subplot(2, 2, 4)
plt.plot(history.history['constraint_violation_ratio'])
plt.plot(history.history['val_constraint_violation_ratio'])
plt.title('Constraint Violation Ratio')
plt.ylabel('Ratio')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.savefig(f"{inter_model_outWorkspace}/training_history_{timestamp}.png")
plt.show()

In [ ]:
# After model.fit() has completed
f = open(inter_model_outWorkspace + "output.txt", "a")
sys.stdout = f
metrics_to_track = ['val_masked_rmse', 'val_masked_mse', 'val_masked_mae']

print("\n Validation Metric Progression:")
for metric in metrics_to_track:
    values = history.history.get(metric, [])
    if values:
        print(f"{metric}: Start = {values[0]:.4f}, End = {values[-1]:.4f}")
    else:
        print(f"{metric}: Not found in history.")
print(f"Final activation function: {final_activation}")

In [ ]:
# testing out this function with test date
# split up the features and arrarys 
def target_feature_stacks(start_year, end_year, target_splits_path, fSCA_path, vegetation_path, phv_path, extension_filter, desired_shape, debug_output_folder, num_of_channels):
        ## create empty arrays
        years = list(range(start_year, (int(end_year) + 1)))
        featureArray = []
        targetArray = []
        extent_list = []
        crs_list = []
        
        # loop through the years and feature data
        for year in years:
            print(f"Processing {year}")
            targetSplits = target_splits_path
            fSCAWorkspace = fSCA_path
            for sample in os.listdir(targetSplits):
                featureTuple = ()
                featureName = []
                # loop through each sample and get the corresponding features
                if sample.endswith(extension_filter):
                    # read in data
                    with rasterio.open(targetSplits + sample) as samp_src:
                        samp_data = samp_src.read(1)
                        meta = samp_src.meta.copy()
                        samp_extent = samp_src.bounds
                        samp_transform = samp_src.transform
                        samp_crs = samp_src.crs
                        # apply a no-data mask
                        mask = samp_data >= 0
                        msked_target = np.where(mask, samp_data, -1)
                        target_shape = msked_target.shape
            
                        # flatted data
                        samp_flat = msked_target.flatten()
                        
        
                    # try to get the fsca variables 
                    sample_root = "_".join(sample.split("_")[:2])
                    for fSCA in os.listdir(fSCAWorkspace):
                        if fSCA.endswith(extension_filter) and fSCA.startswith(sample_root):
                            featureName.append(f"{fSCA[:-4]}")
                            fsca_norm = read_aligned_raster(src_path=fSCAWorkspace + fSCA, extent=samp_extent, target_shape=target_shape)
                            fsca_norm = min_max_scale(fsca_norm, min_val=0, max_val=100)
                            fsca_flat = fsca_norm.flatten()
                            fsca_binary_mask = (fsca_flat > 0).astype(np.float32)
                            featureTuple += (fsca_norm,)
                            # print(fsca_norm.shape)
                            if fsca_norm.shape != desired_shape:
                                print(f"WRONG SHAPE FOR {sample}: FSCA")
                                output_debug_path = debug_output_folder + f"/{sample_root}_BAD_FSCA.tif"
                                save_array_as_raster(
                                    output_path=output_debug_path,
                                    array=fsca_norm,
                                    extent=samp_extent,
                                    crs=samp_crs,
                                    nodata_val=-1
                                )
            
                    # get a DOY array into a feature 
                    date_string = sample.split("_")[1]
                    doy_str = date_string[-3:]
                    doy = float(doy_str)
                    DOY_array = np.full_like(msked_target, doy)
                    doy_norm = min_max_scale(DOY_array,  min_val=0, max_val=366)
                    featureTuple += (doy_norm,)
                    featureName.append(doy)
            
                    # get the vegetation array
                    for tree in os.listdir(vegetation_path):
                        if tree.endswith(extension_filter):
                            if tree.startswith(f"{year}"):
                                featureName.append(f"{tree[:-4]}")
                                tree_norm = read_aligned_raster(
                                src_path=tree_workspace + tree,
                                extent=samp_extent,
                                target_shape=target_shape
                                )
                                tree_norm = min_max_scale(tree_norm, min_val=0, max_val=100)
                                featureTuple += (tree_norm,)
                                if tree_norm.shape != desired_shape:
                                    print(f"WRONG SHAPE FOR {sample}: TREE")
                                    output_debug_path = debug_output_folder + f"/{sample_root}_BAD_TREE.tif"
                                    save_array_as_raster(
                                        output_path=output_debug_path,
                                        array=fsca_norm,
                                        extent=samp_extent,
                                        crs=samp_crs,
                                        nodata_val=-1
                                    )
                    
            
                    # # get all the features in the fodler 
                    for phv in os.listdir(phv_path):
                        if phv.endswith(extension_filter):
                            featureName.append(f"{phv[:-4]}")
                            phv_data = read_aligned_raster(src_path=phv_features + phv, extent=samp_extent, target_shape=target_shape)
                            featureTuple += (phv_data,)
                            if phv_data.shape != desired_shape:
                                print(f"WRONG SHAPE FOR {sample}: {phv}")
                                output_debug_path = debug_output_folder + f"/{sample_root}_BAD_{phv[:-4]}.tif"
                                save_array_as_raster(
                                    output_path=output_debug_path,
                                    array=fsca_norm,
                                    extent=samp_extent,
                                    crs=samp_crs,
                                    nodata_val=-1
                                )
                    feature_stack = np.dstack(featureTuple)
                    if feature_stack.shape[2] != num_of_channels:
                        print(f"{sample} has shape {feature_stack.shape} — missing or extra feature?")
                        print(featureName)
                        print(" ")
                    else:
                        featureArray.append(feature_stack)
                        target_combined = np.concatenate([samp_flat, fsca_binary_mask])
                        targetArray.append(target_combined)
                        # targetArray.append(samp_flat)
                        extent_list.append(samp_extent)
                        crs_list.append(samp_crs)
            print("You go girl!")
        print("all data split into target and feature array")
        return  np.array(featureArray), np.array(targetArray), extent_list, crs_list

In [ ]:
# testing out this function with test date
# split up the features and arrarys 
def target_feature_stacks_testGroups(year, target_splits_path, fSCA_path, vegetation_path, phv_path, extension_filter, desired_shape, debug_output_folder, num_of_channels):
        ## create empty arrays
        featureArray = []
        targetArray = []
        extent_list = []
        crs_list = []
        
        # loop through the years and feature data
        # print(f"Processing {group}")
        targetSplits = target_splits_path
        fSCAWorkspace = fSCA_path
        for sample in os.listdir(targetSplits):
            featureTuple = ()
            featureName = []
            # loop through each sample and get the corresponding features
            if sample.endswith(extension_filter):
                # read in data
                with rasterio.open(targetSplits + sample) as samp_src:
                    samp_data = samp_src.read(1)
                    meta = samp_src.meta.copy()
                    samp_extent = samp_src.bounds
                    samp_transform = samp_src.transform
                    samp_crs = samp_src.crs
                    # apply a no-data mask
                    mask = samp_data >= 0
                    msked_target = np.where(mask, samp_data, -1)
                    target_shape = msked_target.shape
        
                    # flatted data
                    samp_flat = msked_target.flatten()
                    
    
                # try to get the fsca variables 
                sample_root = "_".join(sample.split("_")[:2])
                for fSCA in os.listdir(fSCAWorkspace):
                    if fSCA.endswith(extension_filter) and fSCA.startswith(sample_root):
                        featureName.append(f"{fSCA[:-4]}")
                        fsca_norm = read_aligned_raster(src_path=fSCAWorkspace + fSCA, extent=samp_extent, target_shape=target_shape)
                        fsca_norm = min_max_scale(fsca_norm, min_val=0, max_val=100)
                        fsca_flat = fsca_norm.flatten()
                        fsca_binary_mask = (fsca_flat > 0).astype(np.float32)
                        featureTuple += (fsca_norm,)
                        # print(fsca_norm.shape)
                        if fsca_norm.shape != desired_shape:
                            print(f"WRONG SHAPE FOR {sample}: FSCA")
                            output_debug_path = debug_output_folder + f"/{sample_root}_BAD_FSCA.tif"
                            save_array_as_raster(
                                output_path=output_debug_path,
                                array=fsca_norm,
                                extent=samp_extent,
                                crs=samp_crs,
                                nodata_val=-1
                            )
        
                # get a DOY array into a feature 
                date_string = sample.split("_")[1]
                doy_str = date_string[-3:]
                doy = float(doy_str)
                DOY_array = np.full_like(msked_target, doy)
                doy_norm = min_max_scale(DOY_array,  min_val=0, max_val=366)
                featureTuple += (doy_norm,)
                featureName.append(doy)
        
                # get the vegetation array
                for tree in os.listdir(vegetation_path):
                    if tree.endswith(extension_filter):
                        if tree.startswith(f"{year}"):
                            featureName.append(f"{tree[:-4]}")
                            tree_norm = read_aligned_raster(
                            src_path=tree_workspace + tree,
                            extent=samp_extent,
                            target_shape=target_shape
                            )
                            tree_norm = min_max_scale(tree_norm, min_val=0, max_val=100)
                            featureTuple += (tree_norm,)
                            if tree_norm.shape != desired_shape:
                                print(f"WRONG SHAPE FOR {sample}: TREE")
                                output_debug_path = debug_output_folder + f"/{sample_root}_BAD_TREE.tif"
                                save_array_as_raster(
                                    output_path=output_debug_path,
                                    array=fsca_norm,
                                    extent=samp_extent,
                                    crs=samp_crs,
                                    nodata_val=-1
                                )
                
        
                # # get all the features in the fodler 
                for phv in os.listdir(phv_path):
                    if phv.endswith(extension_filter):
                        featureName.append(f"{phv[:-4]}")
                        phv_data = read_aligned_raster(src_path=phv_features + phv, extent=samp_extent, target_shape=target_shape)
                        featureTuple += (phv_data,)
                        if phv_data.shape != desired_shape:
                            print(f"WRONG SHAPE FOR {sample}: {phv}")
                            output_debug_path = debug_output_folder + f"/{sample_root}_BAD_{phv[:-4]}.tif"
                            save_array_as_raster(
                                output_path=output_debug_path,
                                array=fsca_norm,
                                extent=samp_extent,
                                crs=samp_crs,
                                nodata_val=-1
                            )
                feature_stack = np.dstack(featureTuple)
                if feature_stack.shape[2] != num_of_channels:
                    print(f"{sample} has shape {feature_stack.shape} — missing or extra feature?")
                    print(featureName)
                    print(" ")
                else:
                    featureArray.append(feature_stack)
                    # targetArray.append(samp_flat)
                    target_combined = np.concatenate([samp_flat, fsca_binary_mask])
                    targetArray.append(target_combined)
                    extent_list.append(samp_extent)
                    crs_list.append(samp_crs)
        # print("You go girl!")
        # print("all data split into target and feature array")
        return  np.array(featureArray), np.array(targetArray), extent_list, crs_list

In [ ]:
# Create folder
print("Testing Group 1")
os.makedirs(inter_model_outWorkspace + f"outTifs_G1_yPreds_tifs/", exist_ok=True)
outGroup1 = inter_model_outWorkspace + f"outTifs_G1_yPreds_tifs/"

# Load test data
X_train_g1, y_train_g1, g1_train_extents, g1_train_crs = target_feature_stacks_testGroups(
    year=2022, 
    target_splits_path = WorkspaceBase + f"test_groups/Group1/train/", 
    fSCA_path = WorkspaceBase + f"2022/fSCA/", 
    vegetation_path = WorkspaceBase + "treeCover/",  
    phv_path = WorkspaceBase + "features/scaled/", 
    extension_filter = ".tif", 
    desired_shape = (256, 256), 
    debug_output_folder = "./debug_outputs/", 
    num_of_channels = 14
)

X_test_g1, y_test_g1, g1_test_extents, g1_test_crs = target_feature_stacks_testGroups(
    year = 2022,
    target_splits_path = WorkspaceBase + f"test_groups/Group1/test/", 
    fSCA_path = WorkspaceBase + f"2022/fSCA/", 
    vegetation_path = WorkspaceBase + "treeCover/", 
    phv_path = WorkspaceBase + "features/scaled/", 
    extension_filter = ".tif", 
    desired_shape = (256, 256), 
    debug_output_folder = "./debug_outputs/", 
    num_of_channels = 14
)

from tensorflow.keras.models import load_model

# Include ALL custom functions in the custom_objects dictionary
custom_objects = {
    'masked_loss_fn': masked_loss_fn,
    'masked_mse': masked_mse,
    'masked_mae': masked_mae,
    'masked_rmse': masked_rmse,
    'constrained_masked_mse': constrained_masked_mse,
    'constraint_violation_ratio': constraint_violation_ratio
}

# Load model with all custom objects
model = load_model(
    f"{inter_model_outWorkspace}/best_model_{timestamp}.keras", 
    custom_objects=custom_objects
)

# Convert lists to NumPy arrays if they aren't already
X_test_array_g1 = np.array(X_test_g1)
y_test_array_g1 = np.array(y_test_g1)

# Make sure shapes are correct for model input
print(f"X_test shape: {X_test_array_g1.shape}")  # should be (num_samples, height, width, channels)
print(f"y_test shape: {y_test_array_g1.shape}")  # should confirm expected shape

# Check for NaN values that could cause issues
print(f"X contains NaN: {np.isnan(X_test_array_g1).any()}")
print(f"y contains NaN: {np.isnan(y_test_array_g1).any()}")

# Model predictions
y_pred_g1 = model.predict(X_test_array_g1, batch_size=32)
print(f"Predictions shape: {y_pred_g1.shape}")

# Evaluate model
metrics = model.evaluate(X_test_array_g1, y_test_array_g1, batch_size=32, return_dict=True)
print("\nTest Group 1 Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

# Save prediction images
for i, pred in enumerate(y_pred_g1):
    # Ensure proper reshaping based on dimensions
    if len(pred.shape) > 2:
        array = pred.reshape((256, 256))
    else:
        array = pred
        
    # Get correct shape for mask from y_test
    y_test_item = y_test_g1[i]
    if len(y_test_item.shape) > 2:
        # Get just the SWE part if y includes both SWE and fSCA
        half_size = 65536  # 256*256
        if y_test_item.size > half_size:
            y_test_item = y_test_item[:half_size].reshape((256, 256))
        else:
            y_test_item = y_test_item.reshape((256, 256))
    
    # Create mask for valid values
    mask = y_test_item != -1
    
    # Apply mask: set prediction to -1 where original target had no data
    array_masked = np.where(mask, array, -1)
    
    save_array_as_raster(
        output_path=f"{outGroup1}/prediction_{i}.tif",
        array=array_masked.astype(np.float32),
        extent=g1_test_extents[i],
        crs=g1_test_crs[i],
        nodata_val=-1
    )

In [ ]:
# # create folder
# print("Testing Group 1")
# os.makedirs(inter_model_outWorkspace + f"outTifs_G1_yPreds_tifs/", exist_ok=True)
# outGroup1 = inter_model_outWorkspace + f"outTifs_G1_yPreds_tifs/"
# X_train_g1, y_train_g1, g1_train_extents, g1_train_crs = target_feature_stacks_testGroups(year=2022, 
#                                                    target_splits_path = WorkspaceBase + f"test_groups/Group1/train/", 
#                                                    fSCA_path = WorkspaceBase + f"2022/fSCA/", 
#                                                    vegetation_path = WorkspaceBase + "treeCover/",  
#                                                    phv_path = WorkspaceBase + "features/scaled/", 
#                                                    extension_filter = ".tif", 
#                                                    desired_shape = (256, 256), 
#                                                    debug_output_folder = "./debug_outputs/", 
#                                                    num_of_channels = 14)
# X_test_g1, y_test_g1, g1_test_extents, g1_test_crs = target_feature_stacks_testGroups(year = 2022,
#                                                target_splits_path = WorkspaceBase + f"test_groups/Group1/test/", 
#                                                fSCA_path = WorkspaceBase + f"2022/fSCA/", 
#                                                vegetation_path = WorkspaceBase + "treeCover/", 
#                                                phv_path = WorkspaceBase + "features/scaled/", 
#                                                extension_filter = ".tif", 
#                                                desired_shape = (256, 256), 
#                                                debug_output_folder = "./debug_outputs/", 
#                                                num_of_channels = 14)

# from tensorflow.keras.models import load_model

# # Custom loss function must be re-declared if you used one
# model = load_model(f"{inter_model_outWorkspace}/best_model_{timestamp}.keras", custom_objects={
#     'loss': masked_rmse
# })

# # Convert lists to NumPy arrays if they aren't already
# X_test_array_g1 = np.array(X_test_g1)
# y_test_array_g1 = np.array(y_test_g1)

# # Make sure shapes are correct for model input
# print(f"X_test shape: {X_test_array_g1.shape}")  # should be (num_samples, 256, 256, num_channels)
# print(f"y_test shape: {y_test_array_g1.shape}")  # should be (num_samples, 256, 256, 1) or (num_samples, 256, 256)

# # model predictions
# y_pred_g1 = model.predict(X_test_array_g1, batch_size=32)

# # test loss
# loss = model.evaluate(X_test_array_g1, y_test_array_g1, batch_size=32)

# for i, pred in enumerate(y_pred_g1):
#     array = pred.reshape((256, 256))
#     mask = y_test_g1[i].reshape((256, 256)) != -1  # True where data is valid
    
#     # Apply mask: set prediction to -1 where original target had no data
#     array_masked = np.where(mask, array, -1)
#     save_array_as_raster(
#         output_path=f"{outGroup1}/prediction_{i}.tif",
#         array=array_masked.astype(np.float32),
#         extent=g1_test_extents[i],
#         crs=g1_test_crs[i],
#         nodata_val=-1
#     )

# # Print test loss and metrics
# metrics = model.evaluate(X_test_array_g1, y_test_array_g1, batch_size=32, return_dict=True)
# print("\n Test Group 1 Metrics:")
# print(f"Masked MSE (Loss): {metrics['loss']:.4f}")
# print(f"Masked RMSE:       {metrics['masked_rmse']:.4f}")
# print(f"Masked MAE:        {metrics['masked_mae']:.4f}")
# print(f"Masked MSE Metric: {metrics['masked_mse']:.4f}")

In [ ]:
# create folder
print("\n Testing Group 2")
os.makedirs(inter_model_outWorkspace + f"outTifs_G2_yPreds_tifs/", exist_ok=True)
outGroup2 = inter_model_outWorkspace + f"outTifs_G2_yPreds_tifs/"

X_train_g2, y_train_g2, g2_train_extents, g2_train_crs = target_feature_stacks_testGroups(year=2023, 
                                                   target_splits_path = WorkspaceBase + f"test_groups/Group2/train/", 
                                                   fSCA_path = WorkspaceBase + f"2023/fSCA/", 
                                                   vegetation_path = WorkspaceBase + "treeCover/",  
                                                   phv_path = WorkspaceBase + "features/scaled/", 
                                                   extension_filter = ".tif", 
                                                   desired_shape = (256, 256), 
                                                   debug_output_folder = "./debug_outputs/", 
                                                   num_of_channels = 14)
X_test_g2, y_test_g2, g2_test_extents, g2_test_crs = target_feature_stacks_testGroups(year = 2023,
                                               target_splits_path = WorkspaceBase + f"test_groups/Group2/test/", 
                                               fSCA_path = WorkspaceBase + f"2023/fSCA/", 
                                               vegetation_path = WorkspaceBase + "treeCover/", 
                                               phv_path = WorkspaceBase + "features/scaled/", 
                                               extension_filter = ".tif", 
                                               desired_shape = (256, 256), 
                                               debug_output_folder = "./debug_outputs/", 
                                               num_of_channels = 14)

from tensorflow.keras.models import load_model

# Custom loss function must be re-declared if you used one
model = load_model(f"{inter_model_outWorkspace}/best_model_{timestamp}.keras", custom_objects={
    'loss': masked_rmse
})


# Convert lists to NumPy arrays if they aren't already
X_test_array_g2 = np.array(X_test_g2)
y_test_array_g2 = np.array(y_test_g2)

# Make sure shapes are correct for model input
print(f"X_test shape: {X_test_array_g2.shape}")  # should be (num_samples, 256, 256, num_channels)
print(f"y_test shape: {y_test_array_g2.shape}")  # should be (num_samples, 256, 256, 1) or (num_samples, 256, 256)

# model predictions
y_pred_g2 = model.predict(X_test_array_g2, batch_size=32)

# test loss
loss = model.evaluate(X_test_array_g2, y_test_array_g2, batch_size=32)

for i, pred in enumerate(y_pred_g2):
    array = pred.reshape((256, 256))
    mask = y_test_g2[i].reshape((256, 256)) != -1  # True where data is valid
    
    # Apply mask: set prediction to -1 where original target had no data
    array_masked = np.where(mask, array, -1)
    save_array_as_raster(
        output_path=f"{outGroup2}/prediction_{i}.tif",
        array=array_masked.astype(np.float32),
        extent=g2_test_extents[i],
        crs=g2_test_crs[i],
        nodata_val=-1
    )
# Print test loss and metrics
metrics = model.evaluate(X_test_array_g2, y_test_array_g2, batch_size=32, return_dict=True)
print("\n Test Group 2 Metrics:")
print(f"Masked MSE (Loss): {metrics['loss']:.4f}")
print(f"Masked RMSE:       {metrics['masked_rmse']:.4f}")
print(f"Masked MAE:        {metrics['masked_mae']:.4f}")
print(f"Masked MSE Metric: {metrics['masked_mse']:.4f}")

In [ ]:
# create folder
print("\n Testing Group 3")
os.makedirs(inter_model_outWorkspace + f"outTifs_G3_yPreds_tifs/", exist_ok=True)
outGroup3 = inter_model_outWorkspace + f"outTifs_G3_yPreds_tifs/"

X_train_g3, y_train_g3, g3_train_extents, g3_train_crs = target_feature_stacks_testGroups(year=2024, 
                                                   target_splits_path = WorkspaceBase + f"test_groups/Group3/train/", 
                                                   fSCA_path = WorkspaceBase + f"2024/fSCA/", 
                                                   vegetation_path = WorkspaceBase + "treeCover/",  
                                                   phv_path = WorkspaceBase + "features/scaled/", 
                                                   extension_filter = ".tif", 
                                                   desired_shape = (256, 256), 
                                                   debug_output_folder = "./debug_outputs/", 
                                                   num_of_channels = 14)
X_test_g3, y_test_g3, g3_test_extents, g3_test_crs = target_feature_stacks_testGroups(year = 2024,
                                               target_splits_path = WorkspaceBase + f"test_groups/Group3/test/", 
                                               fSCA_path = WorkspaceBase + f"2024/fSCA/", 
                                               vegetation_path = WorkspaceBase + "treeCover/", 
                                               phv_path = WorkspaceBase + "features/scaled/", 
                                               extension_filter = ".tif", 
                                               desired_shape = (256, 256), 
                                               debug_output_folder = "./debug_outputs/", 
                                               num_of_channels = 14)

from tensorflow.keras.models import load_model

# Custom loss function must be re-declared if you used one
model = load_model(f"{inter_model_outWorkspace}/best_model_{timestamp}.keras", custom_objects={
    'loss': masked_rmse
})


# Convert lists to NumPy arrays if they aren't already
X_test_array_g3 = np.array(X_test_g3)
y_test_array_g3 = np.array(y_test_g3)

# Make sure shapes are correct for model input
print(f"X_test shape: {X_test_array_g3.shape}")  # should be (num_samples, 256, 256, num_channels)
print(f"y_test shape: {y_test_array_g3.shape}")  # should be (num_samples, 256, 256, 1) or (num_samples, 256, 256)

# model predictions
y_pred_g3 = model.predict(X_test_array_g3, batch_size=32)

# test loss
loss = model.evaluate(X_test_array_g3, y_test_array_g3, batch_size=32)

for i, pred in enumerate(y_pred_g3):
    array = pred.reshape((256, 256))
    mask = y_test_g3[i].reshape((256, 256)) != -1  # True where data is valid
    
    # Apply mask: set prediction to -1 where original target had no data
    array_masked = np.where(mask, array, -1)
    save_array_as_raster(
        output_path=f"{outGroup3}/prediction_{i}.tif",
        array=array_masked.astype(np.float32),
        extent=g3_test_extents[i],
        crs=g3_test_crs[i],
        nodata_val=-1
    )

# Print test loss and metrics
metrics = model.evaluate(X_test_array_g3, y_test_array_g3, batch_size=32, return_dict=True)

print("\n Test Group 3 Metrics:")
print(f"Masked MSE (Loss): {metrics['loss']:.4f}")
print(f"Masked RMSE:       {metrics['masked_rmse']:.4f}")
print(f"Masked MAE:        {metrics['masked_mae']:.4f}")
print(f"Masked MSE Metric: {metrics['masked_mse']:.4f}")

In [ ]:
end_time = time.time()
total_seconds = end_time - start_time
total_minutes = total_seconds / 60
print(f"Training time: {total_minutes:.2f} minutes")